# 2) Find the duplicates!

For answering this question we have downloaded text [**passwords2.txt**](https://drive.google.com/open?id=1wTmOU-yqk4qdQYg42AquhzgpNGrRA96d). Then to answer the first part of the question we have to define a hash function to answer this questions. **Find duplicate strings**

But for the first part of this question we have to follow the definition of duplicates where order does not matter e.g  "AABA" = "AAAB" and "AAB"!= ABB

There are three steps we have follow. 
1. Convert the string containing the password to a (potentially large) number
2. Use a hash function to map the number to a large range. Read the class material and search the internet for this part (but you need to write the code yourself).
3. After having defined the hash function, find if two numbers fall on the same range

In [2]:
import pandas as pd # imported pandas
import gmpy2
import sympy

In [3]:
# use chunk size 9999999 lines
c_size = 9999999
txt_file = "passwords2.txt" #text file name

For simple comparison without considering order we really have to define very simple function as simple as it should be possible otherwise we will not have result as wanted. 

In [4]:
def simple_hash_function(passw, table_size):
    number = 0
    for character in passw: # getting each character of the password
        number += ord(character) # getting aschi of the character
    return number % table_size # returns the modular result

In [5]:
table_size_to_pass = 11 # should be prime for optimal result
first_string = "AABA" 
second_string ="AAAB"
first_value = simple_hash_function(first_string, table_size_to_pass)
second_value = simple_hash_function(second_string, table_size_to_pass)

In [6]:
[first_value,second_value]

[8, 8]

As you can see above it *first_value* and *second_value* are the same. 

#### In the file there are 10M duplicates, can you detect them?

To asnwer this question I have to read the file which is 2.2G which will take so much memory and of - course we can detect them but right now there is very high chance for getting collision we will resolve this problem in second part of the question where we consider the order as well. 

In [43]:
def open_file_return_passwords(file_name, number_of_lines = 10000, starting_index = 0, consider_size = True):
    i = 0
    p_items = []
    if(starting_index > 0):
        number_of_lines +=  starting_index
    c_size = number_of_lines
    for line in open(txt_file):
        if i == c_size and consider_size:
            break
        password = line.strip()
        if not starting_index > i:
            p_items.append(password)
        i += 1
    return p_items

Now we know how to read the file, lets move to answering the question detecting duplicate for this we need the Hash Table which size should be prime number for resolving performance issue. For the actual size as I am converting aschi characters to of 20 length passowrd. Each aschi-character length should not be greater than 127 as all the characters number is below this. 

In [8]:
table_size = ord('z') *20*31   # considering we have 20 character and I have multiplied it prim number so that 
table_size = sympy.nextprime(table_size) # getting next prim number from sympy library

In [23]:
hash_table =[] # we will use chaining technique for hash table values
passwords = open_file_return_passwords(txt_file, c_size)# open_file_and_return_passwords(txt_file, c_size)

for i in range(table_size):
    hash_table.append({}) # initializing table items
i = 0

for password in passwords:
    table_index = simple_hash_function(password, table_size)
    password = ''.join(sorted(password)) ## ignoring order of the password
    if password not in hash_table[table_index].keys():
        hash_table[table_index][password] = 0
    hash_table[table_index][password] = hash_table[table_index][password] + 1
    i = i+1

After running above code we have everything we need to answer the questions but ofcourse we are only considering first **9999999** lines of *passwords2.txt* file. Lets find duplicates first.  

In [24]:
#number of duplicates 
duplicates = 0 # this duplicate consider that we have total duplicate item adding both number of duplicates 
# which can be more than 1 
for item in hash_table:
    i = 0
    for key, value in item.items(): # dictionary item should have only one item otherwise they are false positive
        if(value > 1):
            duplicates += (value - 1) # removing 1 if only one count then that is not duplicate
duplicates

0

There are no duplicates in above **9999999** lines of the values. 

###### Does it happen that two strings with different characters are hashed to the same value? If yes, could you provide the number of False Positive?

Yes, it depends on size of the hash table. If size is less than number of passwords or anything that we are checking than there must be false positive. Lets find out how many false positive we have in our **9999999** of hash table size. 

In [25]:
table_size

75641

From above we have **75641** size of the table and we are using chaining technique so we can detect that by checking how many items are false positive. 

In [27]:
#number of duplicates 
false_positive = 0 # this duplicate consider that we have total duplicate item adding both number of duplicates 
# which can be more than 1 
for item in hash_table:
    i = 0
    if (len(item.items()) > 1):
        false_positive += (len(item.items()) - 1)  # removing 1 if only one count then that is not duplicate
false_positive

9999063

There are so many false positive values which says that our hash function sucks. :P Now the total false postive values are 9999063 which says that our function is almost 

In [28]:
c_size-false_positive

936

Only **936** times correct. 

Now lets move to define hash function. Here we are going to use the Multiplicative hash function. The two functions that works best and not too much complex are Multiplicative hash functions. Bernstein's function Kernighan and Ritchie's function. In which **Bernstein's function** uses *INITIAL_VALUE* of *5381* and *M* of *33*; **Kernighan and Ritchie's** function uses *INITIAL_VALUE* of 0 and *M* of 31. Here I am going to choose Bernstein's function. I am going to follow the **Bernstein's function**

In [29]:
def hash_function_with_weighting(password, table_size, INITIAL_VALUE = 5381, M = 33 ): 
    hash_value = INITIAL_VALUE;
    m = M
    for i in range(len(password)):
        hash_value =  m * hash_value + i * ord(password[i])
    return hash_value % table_size

Note that both *INITIAL_VALUE* and *M* is prime number. And we will consider the prime number as well for considering our *Hash Table* size or *array* size. 

Lets answer the first question that **find if two numbers fall on the same range*

In [32]:
table_size_to_pass = 11 # should be prime for optimal result
first_string = "AABA" 
second_string ="AAAB"
first_value = hash_function_with_weighting(first_string, table_size_to_pass)
second_value = hash_function_with_weighting(second_string, table_size_to_pass)

In [34]:
[first_value,second_value]

[8, 0]

As you can see that obove both strings are not equal. 

#### In the file there are 10M duplicates, can you detect them?

To answer this question now we also considering order now lets whether it will resolve issues with which we have so many false positives. We will also consider the same size of hash table. Lets see..

In [49]:
hash_table =[] # we will use chaining technique for hash table values
starting_index=99999999
passwords = open_file_return_passwords(txt_file, c_size,starting_index)# open_file_and_return_passwords(txt_file, c_size)

for i in range(table_size):
    hash_table.append({}) # initializing table items
i = 0

for password in passwords:
    table_index = hash_function_with_weighting(password, table_size)
    if password not in hash_table[table_index].keys():
        hash_table[table_index][password] = 0
    hash_table[table_index][password] = hash_table[table_index][password] + 1
    i = i+1

In [50]:
passwords[:3]

["8SyBU',YDgw*.ZAC#j4P", 'OHcv-/U3QI$rdqYTef"D', 'QtA*.xM$e(+"aO36r&Uo']

After running above code we have everything we need to answer the questions but ofcourse we are only considering first **9999999** lines of *passwords2.txt* file. Lets find duplicates first. But duplicates in first 99999999 lines is never going to be appear as it was before so I have sent starting index. 

In [51]:
#number of duplicates 
duplicates = 0 # this duplicate consider that we have total duplicate item adding both number of duplicates 
# which can be more than 1 
for item in hash_table:
    i = 0
    for key, value in item.items(): # dictionary item should have only one item otherwise they are false positive
        if(value > 1):
            duplicates += (value - 1) # removing 1 if only one count then that is not duplicate
duplicates

0

Still no duplicates in next **99999999** values. 

###### Does it happen that two strings with different characters are hashed to the same value? If yes, could you provide the number of False Positive?

Yes, it depends on size of the hash table. If size is less than number of passwords or anything that we are checking than there must be false positive. Lets find out how many false positive we have in our **9999999** of hash table size. 

In [52]:
table_size

75641

From above we have **75641** size of the table and we are using chaining technique so we can detect that by checking how many items are false positive. 

In [53]:
#number of duplicates 
false_positive = 0 # this duplicate consider that we have total duplicate item adding both number of duplicates 
# which can be more than 1 
for item in hash_table:
    i = 0
    if (len(item.items()) > 1):
        false_positive += (len(item.items()) - 1)  # removing 1 if only one count then that is not duplicate
false_positive

9924358

Now you can see that false positives are drastically decreased. 

In [54]:
c_size-false_positive

75641

Now our function is **75641** times correct.  

No hash function is perfect enough to lower the false positive. So the best approach for the hashing is to use different hash functions randomly to decreased the chance of the false positives. 